In [0]:
%sql
create table datamodeling.default.scdtype1_source
(
  prod_id INT,
  Prod_name STRING,
  prod_cat STRING,
  processDate Date
)

In [0]:
%sql
INSERT INTO datamodeling.default.scdtype1_source
VALUES
(1,'prod1','cat1',current_date()),
(2,'prod2','cat2',current_date()),
(3,'prod3','cat3',current_date())

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
create table datamodeling.gold.scdtype1_target
(
  prod_id INT,
  Prod_name STRING,
  prod_cat STRING,
  processDate Date
)

In [0]:
%sql
merge into datamodeling.gold.scdtype1_target t
using datamodeling.default.scdtype1_source s
on t.prod_id = s.prod_id
when matched and s.processDate>=t.processDate then update set *
when not matched then insert *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,3,0,0


In [0]:
%sql
select * from datamodeling.gold.scdtype1_target;

prod_id,Prod_name,prod_cat,processDate
1,prod1,cat1,2026-02-05
2,prod2,cat2,2026-02-05
3,prod3,newcat,2026-02-05


In [0]:
%sql
update datamodeling.default.scdtype1_source set prod_cat='newcat' where prod_id=3;

num_affected_rows
1


## **SCD TYPE_2**

In [0]:
%sql
create table datamodeling.default.scdtype2_source
(
  prod_id INT,
  Prod_name STRING,
  prod_cat STRING,
  processDate Date
)

In [0]:
%sql
INSERT INTO datamodeling.default.scdtype2_source
VALUES
(1,'prod1','cat1',current_date()),
(2,'prod2','cat2',current_date()),
(3,'prod3','cat3',current_date())

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
create table datamodeling.gold.scdtype2_target
(
  prod_id INT,
  Prod_name STRING,
  prod_cat STRING,
  processDate Date,
  start_date Date,
  end_date Date,
  is_current STRING
)

In [0]:
%sql
select *,
        current_timestamp() as start_date,
        cast('3000-01-01' AS timestamp) as end_date,
        'Y' as is_current
from datamodeling.default.scdtype2_source;

prod_id,Prod_name,prod_cat,processDate,start_date,end_date,is_current
1,prod1,cat1,2026-02-05,2026-02-05T14:20:15.049Z,3000-01-01T00:00:00.000Z,Y
2,prod2,cat2,2026-02-05,2026-02-05T14:20:15.049Z,3000-01-01T00:00:00.000Z,Y
3,prod3,newcat,2026-02-05,2026-02-05T14:20:15.049Z,3000-01-01T00:00:00.000Z,Y


In [0]:
%sql
merge into datamodeling.gold.scdtype2_target t
using (
select *,
        current_timestamp() as start_date,
        cast('3000-01-01' AS timestamp) as end_date,
        'Y' as is_current
from datamodeling.default.scdtype2_source
) s
on t.prod_id = s.prod_id and t.is_current='Y'

-- when we have new data with update
when matched and (s.prod_cat<>t.prod_cat or 
s.prod_name<>t.prod_name or s.processDate<>t.processDate) then update set t.end_date=current_date(),
t.is_current='N'

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
%sql
merge into datamodeling.gold.scdtype2_target t
using (
select *,
        current_timestamp() as start_date,
        cast('3000-01-01' AS timestamp) as end_date,
        'Y' as is_current
from datamodeling.default.scdtype2_source
) s
on t.prod_id = s.prod_id and t.is_current='Y'

when not matched then insert(prod_id,
  Prod_name,
  prod_cat,
  processDate,
  start_date,
  end_date,
  is_current)
  VALUES
  (
        s.prod_id,
        s.prod_name,
        s.prod_cat,
        s.processDate,
        s.start_date,
        s.end_date,
        s.is_current
  );

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql
select * from datamodeling.gold.scdtype2_target

prod_id,Prod_name,prod_cat,processDate,start_date,end_date,is_current
1,prod1,cat1,2026-02-05,2026-02-05,3000-01-01,Y
2,prod2,cat2,2026-02-05,2026-02-05,3000-01-01,Y
3,prod3,cat3,2026-02-05,2026-02-05,2026-02-05,N
3,prod3,newcat,2026-02-05,2026-02-05,3000-01-01,Y


In [0]:
%sql
update datamodeling.default.scdtype2_source set prod_cat='newcat' where prod_id=3;

num_affected_rows
1
